In [ ]:
from tifffile import imread, imsave
import os, re, sys, csv
import numpy as np
import matplotlib.pyplot as pyp
from skimage.morphology import remove_small_objects, binary_closing
from skimage.segmentation import find_boundaries
from skimage.measure import regionprops, regionprops_table, label
from skimage.segmentation import clear_border
import cv2
import copy
import pandas as pd
from scipy import ndimage as ndi
import napari
import scanpy as sc
import seaborn as sns
import math
sys.path.append('~/3D_IMC_paper/Python/python_3d_imc_tools')
from io_files import image_filepath_for_3D_stack


In [ ]:
## function from skimage package https://github.com/scikit-image/scikit-image/blob/main/skimage/measure/_regionprops.py#L869-L1161

COL_DTYPES = {
    'area': int,
    'bbox': int,
    'bbox_area': int,
    'moments_central': float,
    'centroid': float,
    'convex_area': int,
    'convex_image': object,
    'coords': object,
    'eccentricity': float,
    'equivalent_diameter': float,
    'euler_number': int,
    'extent': float,
    'feret_diameter_max': float,
    'filled_area': int,
    'filled_image': object,
    'moments_hu': float,
    'image': object,
    'inertia_tensor': float,
    'inertia_tensor_eigvals': float,
    'intensity_image': object,
    'label': int,
    'local_centroid': float,
    'major_axis_length': float,
    'max_intensity': int,
    'mean_intensity': float,
    'min_intensity': int,
    'minor_axis_length': float,
    'moments': float,
    'moments_normalized': float,
    'orientation': float,
    'perimeter': float,
    'slice': object,
    'solidity': float,
    'weighted_moments_central': float,
    'weighted_centroid': float,
    'weighted_moments_hu': float,
    'weighted_local_centroid': float,
    'weighted_moments': float,
    'weighted_moments_normalized': float
}

OBJECT_COLUMNS = {
    'image', 'coords', 'convex_image', 'slice',
    'filled_image', 'intensity_image'
}

def  skimage_props_to_dict(regions, properties=('label', 'bbox'), separator='-'):
    """Convert image region properties list into a column dictionary."""

    out = {}
    n = len(regions)
    for prop in properties:
        r = regions[0]
        rp = getattr(r, prop)
        if prop in COL_DTYPES:
            dtype = COL_DTYPES[prop]
        else:
            func = r._extra_properties[prop]
            dtype = _infer_regionprop_dtype(
                func,
                intensity=r._intensity_image is not None,
                ndim=r.image.ndim,
            )
        column_buffer = np.zeros(n, dtype=dtype)

        # scalars and objects are dedicated one column per prop
        # array properties are raveled into multiple columns
        # for more info, refer to notes 1
        if np.isscalar(rp) or prop in OBJECT_COLUMNS or dtype is np.object_:
            for i in range(n):
                column_buffer[i] = regions[i][prop]
            out[prop] = np.copy(column_buffer)
        else:
            if isinstance(rp, np.ndarray):
                shape = rp.shape
            else:
                shape = (len(rp),)

            for ind in np.ndindex(shape):
                for k in range(n):
                    loc = ind if len(ind) > 1 else ind[0]
                    column_buffer[k] = regions[k][prop][loc]
                modified_prop = separator.join(map(str, (prop,) + ind))
                out[modified_prop] = np.copy(column_buffer)
    return out

In [ ]:
def load_channel_stack_for_napari(channel_name_to_load, base_folder, missing, crop_im = True):
    metal_folder = base_folder +"/" + channel_name_to_load
    image_path1 = image_filepath_for_3D_stack(metal_folder)
    image1 = imread(image_path1, pattern = None)
    
    if missing is not None:
        missing_slice_image = np.mean( np.array([image1[missing-1, :,:],image1[missing+1,:,:]]), axis=0)
        image1 =  np.insert(image1,missing, missing_slice_image, axis=0)
    
    for i in range(image1.shape[0]):
        #percent99 = np.percentile(image1[i, :,:], 99)
        #tmp_im = np.clip(image1[i, :,:],0,percent99)
        tmp_im = cv2.normalize(image1[i, :,:], None, alpha=0, beta=65535, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_16U)
        tmp_im = np.clip(tmp_im,0,65535)
        image1[i, :,:] = cv2.GaussianBlur(tmp_im,(3,3),1)
        #image1[i, :,:] = cv2.blur(tmp_im,(3,3))
               
    if crop_im == True:
         image1 = image1[:, y_start:y_end,x_start:x_end]
     
    print('Max pixel value:', np.max(image1))
    print('Median pixel value:', np.percentile(image1, 50))
    return image1

Next two functions enable measurment of Euclidean distance to closest blood vessel mask pixel using a bounding box to avoid needless calculations by checking first if inside a small bouding box there are any blood vessel mask pixels, if not the bouding box is extened. 

In [ ]:
def cell_distance_to_mask_3d (centroid_dict, bounding_box, mask_image):
    z_edge = mask_image.shape[0]
    y_edge = mask_image.shape[1]
    x_edge = mask_image.shape[2]

    min_distance_dict = {}

    for key in centroid_dict:
        z = centroid_dict[key]['centroid-0']
        y = centroid_dict[key]['centroid-1']
        x = centroid_dict[key]['centroid-2']        

        for delta in bounding_box:

            beginning_box_z = max(0, int(z)-delta)
            end_box_z = min(z_edge, int(z)+delta)
            beginning_box_y = max(0, int(y)-delta)
            end_box_y = min(y_edge, int(y)+delta)
            beginning_box_x = max(0, int(x)-delta)
            end_box_x = min(x_edge, int(x)+delta)

            z_mask, y_mask, x_mask = mask_image[beginning_box_z:end_box_z, beginning_box_y:end_box_y, beginning_box_x:end_box_x].nonzero()

            potential_vessel_pixels = np.count_nonzero(z_mask)+ np.count_nonzero(y_mask)+ np.count_nonzero(x_mask)
            if potential_vessel_pixels==0:
                continue
            else:                                                                                             
                min_distance = 1000000

                for i in range(len(y_mask)):
                    distance  = math.sqrt(((z_mask[i]+beginning_box_z)-z)**2 + ((y_mask[i] + beginning_box_y)-y)**2 + ((x_mask[i]+beginning_box_x)-x)**2)
                    if distance < min_distance:
                        min_distance = distance

                if min_distance > delta:
                    continue
                else:
                    min_distance_dict[centroid_dict[key]['label']] = min_distance

            break

    return min_distance_dict
    

In [ ]:
def cell_distance_to_mask_2d (centroid_dict, bounding_box, mask_image):
    z_edge = mask_image.shape[0]
    y_edge = mask_image.shape[1]
    min_distance_dict = {}

    for key in centroid_dict:
        z = centroid_dict[key]['centroid-0']
        y = centroid_dict[key]['centroid-1']

        for delta in bounding_box:

            beginning_box_z = max(0, int(z)-delta)
            end_box_z = min(z_edge, int(z)+delta)
            beginning_box_y = max(0, int(y)-delta)
            end_box_y = min(y_edge, int(y)+delta)

            z_mask, y_mask = mask_image[beginning_box_z:end_box_z, beginning_box_y:end_box_y].nonzero()

            potential_vessel_pixels = np.count_nonzero(z_mask)+ np.count_nonzero(y_mask)
            if potential_vessel_pixels==0:
                continue
            else:                                                                                             
                min_distance = 1000000

                for i in range(len(y_mask)):
                    distance  = math.sqrt(((z_mask[i]+beginning_box_z)-z)**2 + ((y_mask[i] + beginning_box_y)-y)**2)
                    if distance < min_distance:
                        min_distance = distance

                if min_distance > delta:
                    continue
                else:
                    min_distance_dict[centroid_dict[key]['label']] = min_distance

            break

    return min_distance_dict
    

### Set inputs

In [ ]:
# INPUT: single chanel TIFFs from the whole 3D model to use for 

#folder for registeration i.e an image per slice
input_base = '~/3D_model201710/3D_registred_tiffs/IMC_fullStack_registred/imageJ_registration/full_model_aligned/'
vessel_mask_input = input_base + 'blood_vessel_mask.tif'
cell_labels_input = input_base + "measured_mask_final_segmentation_hwatershed_500.00_90%.tif"
bounding_box_size =[1, 5, 10, 30, 50, 80, 100, 120, 150, 200, 250, 300, 400, 500, 1000]

results_file = input_base +'model201710_singleCell_analysis.h5ad'  # the file that will store the analysis results


In [ ]:
vessel_mask = imread(vessel_mask_input)
cell_labels = imread(cell_labels_input)

In [ ]:
adata = sc.read_h5ad(results_file)

In [ ]:
vessel_mask_update = binary_closing(vessel_mask)
vessel_mask_labeled = label(vessel_mask_update)
vessel_mask_labeled = remove_small_objects(vessel_mask_labeled, 60)
vessel_mask_labeled[vessel_mask_labeled>0]=1

In [ ]:
final_vessel_mask = copy.deepcopy(vessel_mask_labeled)

In [ ]:
boundaries_only = np.zeros(cell_labels.shape, dtype = cell_labels.dtype)
k = 0 

while k < boundaries_only.shape[0]: 
    slice_2D = cell_labels[k, :,:]
    boundaries_only[k,:,:] = find_boundaries(slice_2D, connectivity=1, mode='outer', background=0)
    k  = k + 1
cell_labels_with_boundry =np.multiply(np.logical_not(boundaries_only),cell_labels)   

In [ ]:
vessel = imread(input_base + "CD31vWF_ROI_image.tif")


In [ ]:
scaling_factors = [2,1,1]
with napari.gui_qt():
    viewer = napari.view_image(vessel, name = 'vessel', scale = scaling_factors)
    viewer.add_image(vessel_mask_labeled, name = 'vessel_mask_update', scale = scaling_factors)

#### 3D distance calculation to closest blood vessel

In [ ]:
object_diameter_im=regionprops(cell_labels)
object_centroid_dict =dict()
object_centroid_dict = skimage_props_to_dict(object_diameter_im, properties=['label','centroid'])
    
centroid_table = pd.DataFrame.from_dict(object_centroid_dict)
centroid_dict_3d = centroid_table.to_dict('index')

In [ ]:
minimum_distance_dict = cell_distance_to_mask_3d(centroid_dict_3d,bounding_box_size,final_vessel_mask)

In [ ]:
distance_table_3D = pd.DataFrame.from_dict(data = minimum_distance_dict,orient = 'index',dtype=None, columns= ['min_distance'])
distance_table_name = input_base + "distance_to_vessel_3D.csv"
distance_table_3D.to_csv(distance_table_name)

#### Calculate distance for an image labeled as 33 separately ie 2D distance to mask on a specific slice

In [ ]:
object_diameter_im=regionprops(cell_labels[32,:,:])
object_centroid_dict =dict()
object_centroid_dict = skimage_props_to_dict(object_diameter_im, properties=['label','centroid'])
    
centroid_table = pd.DataFrame.from_dict(object_centroid_dict)
centroid_dict_2d_s71 = centroid_table.to_dict('index')

In [ ]:
final_vessel_mask_slice71 = final_vessel_mask[32,:,:]

In [ ]:
minimum_distance_dict = cell_distance_to_mask_2d(centroid_dict_2d_s71,bounding_box_size,final_vessel_mask_slice71)

In [ ]:
distance_table_2D = pd.DataFrame.from_dict(data = minimum_distance_dict,orient = 'index',dtype=None, columns= ['min_distance'])
distance_table_name = input_base + "distance_to_vessel_2D_slice_33.csv"
distance_table_2D.to_csv(distance_table_name)

In [ ]:
cell_labels_with_boundry_2d = cell_labels_with_boundry[32, :, :]

distance_image = np.zeros(cell_labels_with_boundry_2d.shape)
for key in minimum_distance_dict.keys():
    distance_image[cell_labels_with_boundry_2d == key] = round(minimum_distance_dict[key],1)

In [ ]:
max_distance_val = np.max(distance_image)
print(max_distance_val)

In [ ]:
norm_distance_im = distance_image/max_distance_val
norm_distance_im = 1-norm_distance_im
norm_distance_im[norm_distance_im ==1] = 0

In [ ]:
with napari.gui_qt():
    viewer = napari.view_image(norm_distance_im, name = 'vessel')
    viewer.add_image(final_vessel_mask[32,:,:], name = 'vessel_mask_update')

#### Slice 33 T cell and tumor cell distance only

In [ ]:
cluster_labels = list(adata.obs['phenograph'])
object_labels = list(adata.obs['cell_labels'])

cluster_cell_label_dictionary = {}
for item in range(len(object_labels)):
    dict_key = int(cluster_labels[item])
    if dict_key in cluster_cell_label_dictionary.keys():
        cluster_cell_label_dictionary[dict_key].append(int(object_labels[item]))
        
    else:    
        cluster_cell_label_dictionary[dict_key] = []
        cluster_cell_label_dictionary[dict_key].append(int(object_labels[item]))

In [ ]:
#choose clusters to display. Clusters 18 and 13 are for T cells.
t_cell_labels = cluster_cell_label_dictionary[4] + cluster_cell_label_dictionary[6] + cluster_cell_label_dictionary[3] +cluster_cell_label_dictionary[37] + cluster_cell_label_dictionary[2] +cluster_cell_label_dictionary[1] +cluster_cell_label_dictionary[7] +cluster_cell_label_dictionary[5] 
#t_cell_labels = cluster_cell_label_dictionary[18] + cluster_cell_label_dictionary[13]

In [ ]:
cell_labels_with_boundry_2d_subset = cell_labels[32, :, :]
cells_on_slice = np.unique(cell_labels_with_boundry_2d_subset) 
distance_image_subset = np.zeros(cell_labels_with_boundry_2d_subset.shape)


for entry in t_cell_labels:
    if entry in cells_on_slice:
        distance_image_subset[cell_labels_with_boundry_2d_subset == entry] = round(minimum_distance_dict[entry],1)
norm_distance_im = distance_image_subset/max_distance_val
norm_distance_im = 1-norm_distance_im
norm_distance_im[norm_distance_im ==1] = 0

In [ ]:
with napari.gui_qt():
    viewer = napari.view_image(norm_distance_im, name = 'vessel')
    viewer.add_image(final_vessel_mask[32,:,:], name = 'vessel_mask_update')

#### Calculate the distance to vessel mask for each object in 2D - take the min distance if the object occurs on multiple slices

In [ ]:
final_2d_object_distances = {}

for s in range(final_vessel_mask.shape[0]):
    image_2d = final_vessel_mask[s, :, :]
    object_diameter_im=regionprops(cell_labels[s,:,:])
    object_centroid_dict =dict()
    object_centroid_dict_all_2d = skimage_props_to_dict(object_diameter_im, properties=['label','centroid'])
 
    centroid_table = pd.DataFrame.from_dict(object_centroid_dict_all_2d)
    centroid_dict = centroid_table.to_dict('index')
  
    minimum_distance_dict_all_2d = cell_distance_to_mask_2d(centroid_dict,bounding_box_size,image_2d)
    
    for label in minimum_distance_dict_all_2d.keys():
        current_distance = minimum_distance_dict_all_2d[label]
        if label in final_2d_object_distances.keys() : 
            previous_distance = final_2d_object_distances[label]
            if current_distance < previous_distance:
                final_2d_object_distances[label] = current_distance
            else:
                continue
        else:
            final_2d_object_distances[label] = current_distance 

In [ ]:
distance_table_2D = pd.DataFrame.from_dict(data =final_2d_object_distances,orient = 'index',dtype=None, columns= ['min_distance'])
distance_table_name = input_base + "distance_to_vessel_2D_all_cells.csv"
distance_table_2D.to_csv(distance_table_name)

#### For 3D display distances for tumor cells, t cells and all cells for slice 33

In [ ]:
minimum_distance = pd.read_csv(input_base + "distance_to_vessel_3D.csv", index_col = 0)

In [ ]:
minimum_distance_dict = minimum_distance.to_dict('index')

In [ ]:
max_distance_val = 362.9 #normalize to max 2d value ie max distance achieved for all the cells

In [ ]:
cell_labels_with_boundry_2d = cell_labels_with_boundry[32, :, :]

distance_image = np.zeros(cell_labels_with_boundry_2d.shape)
for key in minimum_distance_dict.keys():
    distance_image[cell_labels_with_boundry_2d == key] = round(minimum_distance_dict[key]['min_distance'],1)

In [ ]:
norm_distance_im = distance_image/max_distance_val
norm_distance_im = 1-norm_distance_im
norm_distance_im[norm_distance_im ==1] = 0

In [ ]:
with napari.gui_qt():
    viewer = napari.view_image(norm_distance_im, name = 'vessel')
    viewer.add_image(final_vessel_mask[32,:,:], name = 'vessel_mask_update')

In [ ]:
cluster_labels = list(adata.obs['phenograph'])
object_labels = list(adata.obs['cell_labels'])

cluster_cell_label_dictionary = {}
for item in range(len(object_labels)):
    dict_key = int(cluster_labels[item])
    if dict_key in cluster_cell_label_dictionary.keys():
        cluster_cell_label_dictionary[dict_key].append(int(object_labels[item]))
        
    else:    
        cluster_cell_label_dictionary[dict_key] = []
        cluster_cell_label_dictionary[dict_key].append(int(object_labels[item]))

In [ ]:
#t_cell_labels = cluster_cell_label_dictionary[4] + cluster_cell_label_dictionary[6] + cluster_cell_label_dictionary[3] +cluster_cell_label_dictionary[37] + cluster_cell_label_dictionary[2] +cluster_cell_label_dictionary[1] +cluster_cell_label_dictionary[7] +cluster_cell_label_dictionary[5] 
t_cell_labels = cluster_cell_label_dictionary[18] + cluster_cell_label_dictionary[13] # actual t cells

In [ ]:
cell_labels_with_boundry_2d_subset = cell_labels[32, :, :]
cells_on_slice = np.unique(cell_labels_with_boundry_2d_subset) 
distance_image_subset = np.zeros(cell_labels_with_boundry_2d_subset.shape)


for entry in t_cell_labels:
    if entry in cells_on_slice:
        distance_image_subset[cell_labels_with_boundry_2d_subset == entry] = round(minimum_distance_dict[entry]['min_distance'],1)
norm_distance_im = distance_image_subset/max_distance_val
norm_distance_im = 1-norm_distance_im
norm_distance_im[norm_distance_im ==1] = 0

In [ ]:
with napari.gui_qt():
    viewer = napari.view_image(norm_distance_im, name = 'vessel')
    viewer.add_image(final_vessel_mask[32,:,:], name = 'vessel_mask_update')

#### Distance measurment comparison for 2d and 3d for different cell groups

In [ ]:
minimum_distance = pd.read_csv(input_base + "distance_to_vessel_3D.csv", index_col = 0)
minimum_distance_dict = minimum_distance.to_dict('index')

In [ ]:
minimum_distance_2d = pd.read_csv(input_base + "distance_to_vessel_2D_all_cells.csv", index_col = 0)
minimum_distance_2d = minimum_distance_2d.rename(columns={"min_distance": "min_distance_2d"})
minimum_distance_dict_2d = minimum_distance_2d.to_dict('index')

In [ ]:
united_dict = minimum_distance_dict
for i in minimum_distance_dict.keys():
    if i not in minimum_distance_dict_2d.keys():
        united_dict[i]['min_distance_2d'] = 'NaN'
    else:
        united_dict[i]['min_distance_2d'] = minimum_distance_dict_2d[i]['min_distance_2d']

In [ ]:
distance_table =  pd.DataFrame.from_dict(united_dict,'index' )

In [ ]:
distance_table

#### Plot Distances for all cells

In [ ]:
sns.distplot(distance_table['min_distance'], hist = False, kde = True, label='3D distance')
sns.distplot(distance_table['min_distance_2d'], hist = False, kde = True, label='2D distance')
# Plot formatting
pyp.legend(prop={'size': 12})
pyp.title('Distance to closest blood vessel for all cells')
pyp.xlabel('Distance (um)')
pyp.ylabel('Density') 

#### Plot Distances for T cells

In [ ]:
t_cell_labels = cluster_cell_label_dictionary[18] + cluster_cell_label_dictionary[13]
subset_table = distance_table.filter(items = t_cell_labels, axis = 'index')
sns.distplot(subset_table['min_distance'], hist = False, kde = True, label='3D distance')
sns.distplot(subset_table['min_distance_2d'], hist = False, kde = True, label='2D distance')
# Plot formatting
pyp.legend(prop={'size': 12})
pyp.title('Distance to closest blood vessel for T-cells')
pyp.xlabel('Distance (um)')
pyp.ylabel('Density')  


#### Plot Distances for tumor cells

In [ ]:
t_cell_labels = cluster_cell_label_dictionary[4] + cluster_cell_label_dictionary[6] + cluster_cell_label_dictionary[3] +cluster_cell_label_dictionary[37] + cluster_cell_label_dictionary[2] +cluster_cell_label_dictionary[1] +cluster_cell_label_dictionary[7] +cluster_cell_label_dictionary[5] 
subset_table = distance_table.filter(items = t_cell_labels, axis = 'index')
sns.distplot(subset_table['min_distance'], hist = False, kde = True, label='3D distance')
sns.distplot(subset_table['min_distance_2d'], hist = False, kde = True, label='2D distance')
# Plot formatting
pyp.legend(prop={'size': 12})
pyp.title('Distance to closest blood vessel for luminal epithelial cells')
pyp.xlabel('Distance (um)')
pyp.ylabel('Density')  


#### Plot Distances for basal cells

In [ ]:
t_cell_labels = cluster_cell_label_dictionary[19]
subset_table = distance_table.filter(items = t_cell_labels, axis = 'index')
sns.distplot(subset_table['min_distance'], hist = False, kde = True, label='3D distance')
sns.distplot(subset_table['min_distance_2d'], hist = False, kde = True, label='2D distance')
# Plot formatting
pyp.legend(prop={'size': 12})
pyp.title('Distance to closest blood vessel for basal epithelial cells')
pyp.xlabel('Distance (um)')
pyp.ylabel('Density')  


#### Plot Distances for B cells

In [ ]:
t_cell_labels = cluster_cell_label_dictionary[25] 
subset_table = distance_table.filter(items = t_cell_labels, axis = 'index')
sns.distplot(subset_table['min_distance'], hist = False, kde = True, label='3D distance')
sns.distplot(subset_table['min_distance_2d'], hist = False, kde = True, label='2D distance')
# Plot formatting
pyp.legend(prop={'size': 12})
pyp.title('Distance to closest blood vessel for  B-cells')
pyp.xlabel('Distance (um)')
pyp.ylabel('Density')  


#### Plot Distances for stromal cells

In [ ]:
t_cell_labels = cluster_cell_label_dictionary[17] +  cluster_cell_label_dictionary[22]
subset_table = distance_table.filter(items = t_cell_labels, axis = 'index')
sns.distplot(subset_table['min_distance'], hist = False, kde = True, label='3D distance')
sns.distplot(subset_table['min_distance_2d'], hist = False, kde = True, label='2D distance')
# Plot formatting
pyp.legend(prop={'size': 12})
pyp.title('Distance to closest blood vessel for stromal cells')
pyp.xlabel('Distance (um)')
pyp.ylabel('Density') 

#### Plot Distances for macrophages

In [ ]:
t_cell_labels = cluster_cell_label_dictionary[30]
subset_table = distance_table.filter(items = t_cell_labels, axis = 'index')
sns.distplot(subset_table['min_distance'], hist = False, kde = True, label='3D distance')
sns.distplot(subset_table['min_distance_2d'], hist = False, kde = True, label='2D distance')
# Plot formatting
pyp.legend(prop={'size': 12})
pyp.title('Distance to closest blood vessel for macrophages')
pyp.xlabel('Distance (um)')
pyp.ylabel('Density') 

###### End of notebook